In [14]:
(require circuitous
         esterel-calculus/circuits/compiler
         redex/reduction-semantics
         racket/pretty)

Some possibly helpful laws:

- Posulate 1: `¬( GO ∨ (RES ∧ SEL))` implies all `E'` is `0` and all `kn` are `0`.
- - Correlary 1: This holds for `¬(GO ∨ SEL)`
- - Correlary 2: This holds for `¬(GO ∨ RES)`
- Postulate 2: forall `n` `m`, `m != n`, `¬ (kn ^ km)`, that is at most one `k` wire is selected. (Note: This might be wrong, if GO is given to a selected and resumed pause... this should not be allowed but im not sure...)


In [5]:
(define start
  (circuit
   #:inputs (GO RES rsel r0)
   #:outputs (SEL K0)
   ;; these come from then othing
   (l0 = GO)
   (lsel = false)
   ;; SEL
   (SEL = (or lsel rsel))
   ;; the synchonizer
   (K0 = (and left0 (and right0 both0)))
   (left0 = (or l0 lem))
   (right0 = (or r0 rem))
   (lem = (and SEL (and RES (not lsel))))
   (rem = (and SEL (and RES (not rsel))))
   (both0 = (or l0 r0))))



In [9]:
(assert-same 
 start
 (rename
   #:constraints #f
   (compile-esterel (term (par nothing (nts p 0))))
  '(p-SEL rsel)
  '(pK0 r0)))

In [35]:
(assert-same 
 #:constraints '(and
                 ;; by [sel-defined]
                 (or SEL (not SEL))
                 ;; by  [activation-condition]
                     (implies (not (or GO (and RES SEL)))
                              (not K0)))
 start
             (circuit
              #:inputs (r0 rsel)
              #:outputs (K0 SEL)
              (K0 = r0) (SEL = rsel)))